In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [3]:
batch_size = 4096

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 92600157.34it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 22283976.05it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25257994.22it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17721422.11it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [4]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in:3 , n_in:28 , j_in:1 , s:1 , r_out:5 , n_out:28 , j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:28 , j_in:1 , s:2 , r_out:6 , n_out:14 , j_out:2
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in:6 , n_in:14 , j_in:2 , s:1 , r_out: 10, n_out:14 , j_out:2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in:10 , n_in:14 , j_in:2 , s: 1, r_out: 14, n_out: 14, j_out:2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in:14 , n_in: 14, j_in: 2, s: 2, r_out: 16, n_out: 7, j_out:4
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in: 16, n_in: 7, j_in: 4, s: 1, r_out: 24, n_out: 5, j_out:4
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in:24 , n_in: 5, j_in: 4, s: 1, r_out: 32, n_out: 3, j_out:4
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in: 32, n_in: 3, j_in:4 , s: 1, r_out: 40, n_out: 1, j_out:4
    self.conv7 = nn.Conv2d(1024, 10, 3)
# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)


In [5]:
model = FirstDNN().to(device)

In [6]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-4-57be85d8a5eb>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    # sets the model into training mode. This is necessary to activate certain layers (e.g., dropout or batch normalization) that behave differently during training and inference.
    # dropout layers are activated, and the dropout mask is applied to the input data
    # batch normalization layers normalize the input data using batch statistics
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        # moves the batch of data and its corresponding labels to the GPU
        data, target = data.to(device), target.to(device)
        # clears the gradients of all optimized parameters
        optimizer.zero_grad()
        output = model(data)
        # calculates the negative log-likelihood loss between the model's predictions and the target.
        loss = F.nll_loss(output, target)
        # performs backpropagation to compute the gradients
        loss.backward()
        # updates the model's parameters by taking a step in the direction of the computed gradients.
        optimizer.step()
        # progress bar  update
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


#  model's performance in terms of loss and accuracy
def test(model, device, test_loader):
    # sets the model in evaluation mode.
    # During evaluation, dropout usually turned off and
    # batch normalization using population statistics instead of batch statistics.
    model.eval()
    test_loss = 0
    correct = 0

    # starts a context block where no gradient computations are performed.
    with torch.no_grad():
        for data, target in test_loader:
            # moves the batch of data and its corresponding labels to the GPU
            data, target = data.to(device), target.to(device)
            # Model predicted output
            output = model(data)
            # the negative log-likelihood loss between the predictions and the target.
            # The reduction='sum' argument indicates that the losses should be summed across the batch.
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    # compute the average loss.
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
# Initializing the SGD optimizer.
# The optimizer will be used to update the model's parameters during training.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    # performs the training process, updating the model's parameters based on the optimizer
    train(model, device, train_loader, optimizer, epoch)
    # Evaluate the trained model on the test dataset
    test(model, device, test_loader)

  0%|          | 0/15 [00:00<?, ?it/s]<ipython-input-4-57be85d8a5eb>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=2.290574789047241 batch_id=14: 100%|██████████| 15/15 [00:32<00:00,  2.15s/it]



Test set: Average loss: 2.2884, Accuracy: 4409/10000 (44%)

